In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np

# Классификация отзывов фильмов с IMDB на положительные и отрицательные

<img src="https://github.com/bentrevett/pytorch-sentiment-analysis/raw/bf8cc46e4823ebf9af721b595501ad6231c73632/assets/sentiment1.png">

Общая задача: 

Есть датасет из текстов, необходимо решить задачу классификации на *положительные* отзывы и *отрицательные*

Pipeline:

1. Создать датасеты, разбить данные на обучающие, валидационные, тестовые

2. Для каждого текста - закодировать слова в соответствии со словарём

3. На вход модели подать батч из предложений, то есть массив индексов

4. С помощью embedding превратить индексы - в вектора слов

5. Дальше в модели использовать реккурентную ячейку GRU

6. После этого - линейный слой и софт-макс для классификации 

7. Подсчитать метрику на тестовом датасете

# Работаем с данными

In [2]:
!pip install torchtext
!python -m spacy download en

     |████████████████████████████████| 12.0 MB 5.3 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [3]:
import torch
from torchtext import data
from torchtext.legacy import data

SEED = 2022

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)

In [4]:
from torchtext.legacy import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL, root="./data")

In [5]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

print(vars(train_data.examples[1]))

Number of training examples: 25000
Number of testing examples: 25000
{'text': ['Damon', 'Runyon', "'s", 'world', 'of', 'Times', 'Square', ',', 'in', 'New', 'York', ',', 'prior', 'to', 'its', 'Disneyfication', ',', 'is', 'the', 'basis', 'for', 'this', 'musical', '.', 'Joseph', 'L.', 'Mankiewicz', ',', 'a', 'man', 'who', 'knew', 'about', 'movies', ',', 'directed', 'this', 'nostalgic', 'tribute', 'to', 'the', '"', 'crossroads', 'of', 'the', 'world', '"', 'that', 'show', 'us', 'that', 'underside', 'of', 'New', 'York', 'of', 'the', 'past', '.', 'Frank', 'Loesser', "'s", 'music', 'sounds', 'great', '.', 'We', 'watch', 'a', 'magnificent', 'cast', 'of', 'characters', 'that', 'were', 'typical', 'of', 'the', 'area', '.', 'People', 'at', 'the', 'edges', 'of', 'society', 'tended', 'to', 'gravitate', 'toward', 'that', 'area', 'because', 'of', 'the', 'lights', ',', 'the', 'action', ',', 'the', 'possibilities', 'in', 'that', 'part', 'of', 'town', '.', 'This', 'underbelly', 'of', 'the', 'city', 'made'

In [6]:
# Создаём ещё eval
import random

train_data, valid_data = train_data.split(random_state=random.seed(SEED))

# Создаём словарь
TEXT.build_vocab(train_data, max_size=25000)
LABEL.build_vocab(train_data)

print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

vars(LABEL.vocab)

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


{'freqs': Counter({'neg': 8788, 'pos': 8712}),
 'itos': ['neg', 'pos'],
 'stoi': defaultdict(None, {'neg': 0, 'pos': 1}),
 'unk_index': None,
 'vectors': None}

* stoi (string to int)
* itos (int to string)

In [7]:
print(TEXT.vocab.freqs.most_common(20))
print()
print(TEXT.vocab.itos[:10])
print()
print(LABEL.vocab.stoi)

[('the', 203510), (',', 192440), ('.', 166030), ('and', 109631), ('a', 109528), ('of', 101196), ('to', 93574), ('is', 76094), ('in', 61375), ('I', 55230), ('it', 53864), ('that', 49201), ('"', 44351), ("'s", 43431), ('this', 42349), ('-', 37162), ('/><br', 35748), ('was', 35250), ('as', 30423), ('with', 29965)]

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']

defaultdict(None, {'neg': 0, 'pos': 1})


In [8]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# собираем батчи так, чтобы в каждом батче были примеры наиболее похожей длины
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE,
    device=device,
    sort=True, # Используем сортировку
    sort_key=lambda x: len(x.text))

Посмотрим на какой-нибудь пример из **train_iterator**

In [9]:
train_iterator.data()[0].text

['I',
 'would',
 "n't",
 'rent',
 'this',
 'one',
 'even',
 'on',
 'dollar',
 'rental',
 'night',
 '.']

In [10]:
# Количество использованных слов в словаре
len(TEXT.vocab)

25002

## Создаём модель

In [11]:
class TextPredictionModel(nn.Module):
    def __init__(self, all_words_amount, embedding_size, hidden_size, batch_size=1,
                 n_layers=1, bidirectional_rnn=False, nn_type="RNN", device=torch.device("cpu")):
        super().__init__()
        self.all_words_amount = all_words_amount
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.n_layers = n_layers
        self.device = device
        self.bidirectional_coefficent = 1 if bidirectional_rnn == False else 2
        
        # Embedding принимает два параметра - количество слов в словаре, и размерность представления у каждого слова
        self.embedding = nn.Embedding(all_words_amount, embedding_size)

        # Дальше выбираем тип нейронной сети: RNN или GRU (зависит от параметра nn_type)
        if nn_type == "GRU":
            self.reccurent_cell = nn.GRU(embedding_size, hidden_size, n_layers, bidirectional=bidirectional_rnn)
        else:
            self.reccurent_cell = nn.RNN(embedding_size, hidden_size, n_layers, bidirectional=bidirectional_rnn)

        # Линейный слой с Soft-Max, чтобы в результате получить вероятности принадлежности к каждому классу
        self.predicting = nn.Sequential(nn.Linear(hidden_size * n_layers * self.bidirectional_coefficent, 2),
                                        nn.Softmax())

    def init_hidden(self, batch_size_arg=None):
        return torch.zeros(self.n_layers * self.bidirectional_coefficent,
                           batch_size_arg if batch_size_arg is not None else self.batch_size,
                           self.hidden_size).to(self.device)
    
    def forward(self, X):
        # X - [batch_size, seq_len]
        output = self.embedding(X) # Возвращает представление в виде [embedding_size, seq_size, batch_size]
        output = output.permute(1, 0, 2) # Меняем оси, получаем [seq_size, embedding_size, batch_size]

        # RNN принимает именно в таком порядке input: [seq_size, embedding_size, batch_size], поэтому сделали permute
        # RNN принимает именно в таком порядке hidden_0: [layers_amount, batch_size, hidden_size]
        h_0 = self.init_hidden(X.shape[0])

        # На выходе из RNN - output и hn - то тензор из всех скрытых слоёв, а также последний
        # Мы взяли только последний и назвали его h_n
        h_n = self.reccurent_cell(output, h_0)[1]

        # Меняем вектор h_n - переводим в вид [batch_size, n_layers * hidden * bidirectional_coefficient]
        # То есть для не двухсторонней сети с одним слоем RNN  - просто [batch_size, hidden]
        vector_to_classifier = h_n.view(X.shape[0], -1)

        # Возвращаем результат классификатора
        return self.predicting(vector_to_classifier)

### Device

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Проверим работоспособность модели на искусственном примере

In [13]:
model = TextPredictionModel(9, 10, 12, 4, 2, True, "RNN", device).to(device) 
# all_words_amount, embedding_size, hidden_size, batch_size, n_layers, bidirectional, nn_type, device

X = torch.LongTensor([[0, 1, 2, 3, 4], [1, 2, 3, 0, 0], [2, 3, 1, 5, 0], [0, 6, 4, 7, 1]]).to(device)
result = model(X)
result

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


tensor([[0.4690, 0.5310],
        [0.3758, 0.6242],
        [0.3601, 0.6399],
        [0.4801, 0.5199]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

## Обучение

In [14]:
def accuracy(y, real_y): # Обычная accuracy
    return (1 - torch.abs(torch.max(y, dim=1).indices - real_y).sum() / len(real_y))

In [15]:
ALL_WORD_AMOUNT = len(TEXT.vocab)
EMBEDDING_SIZE = 128
HIDDEN_SIZE = 64
BATCH_SIZE = 64
N_LAYERS = 1
BIDIRECTIONAL = False
NN_TYPE = "GRU"

In [16]:
model = TextPredictionModel(ALL_WORD_AMOUNT, EMBEDDING_SIZE, HIDDEN_SIZE, BATCH_SIZE, N_LAYERS, BIDIRECTIONAL, NN_TYPE, device)

model = model.to(device)
lr = 0.004
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lambda x: 0.95)
loss = torch.nn.CrossEntropyLoss()
epohs = 15

In [19]:
from tqdm.notebook import tqdm

all_losses = []
for epoh in tqdm(range(epohs)):
    c_loss = []
    val_accuracy = []
    train_accuracy = []
    for batch in train_iterator:
        optimizer.zero_grad()
        # Подаём нужные размерности
        output = model(batch.text.permute(1, 0))
        # Считаем лосс
        loss_value = loss(output, batch.label.to(device, dtype=torch.long))
        loss_value.backward()
        
        optimizer.step()
        c_loss.append(loss_value.item())

    # Подсчитываем метрику на валидации и тренировочном датасете
    for batch in valid_iterator:
        val_accuracy.append(accuracy(model(batch.text.permute(1, 0)), batch.label).cpu().numpy())
    for batch in train_iterator:
        train_accuracy.append(accuracy(model(batch.text.permute(1, 0)), batch.label).cpu().numpy())

    # Выводим информацию и делаем шаг scheduler
    print("--" * 15)
    print("EPOCH:\t {}".format(str(epoh)))
    print("Loss:\t\t\t {:7.5f}".format(sum(c_loss) / len(c_loss)))
    print("Learning rate:\t\t {:7.5f}".format(float(optimizer.state_dict()["param_groups"][0]["lr"])))
    print("Accuracy train:\t\t {:7.3%}".format(sum(train_accuracy) / len(train_accuracy)))
    print("Accuracy validation:\t {:7.3%}".format(sum(val_accuracy) / len(val_accuracy)))
    scheduler.step()

  0%|          | 0/15 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


------------------------------
EPOCH:	 0
Loss:			 0.35893
Learning rate:		 0.00380
Accuracy train:		 96.556%
Accuracy validation:	 86.630%
------------------------------
EPOCH:	 1
Loss:			 0.34816
Learning rate:		 0.00361
Accuracy train:		 97.491%
Accuracy validation:	 87.142%
------------------------------
EPOCH:	 2
Loss:			 0.34129
Learning rate:		 0.00343
Accuracy train:		 97.993%
Accuracy validation:	 87.050%
------------------------------
EPOCH:	 3
Loss:			 0.33728
Learning rate:		 0.00326
Accuracy train:		 98.118%
Accuracy validation:	 86.798%
------------------------------
EPOCH:	 4
Loss:			 0.33487
Learning rate:		 0.00310
Accuracy train:		 98.432%
Accuracy validation:	 87.337%
------------------------------
EPOCH:	 5
Loss:			 0.33010
Learning rate:		 0.00294
Accuracy train:		 98.609%
Accuracy validation:	 87.156%
------------------------------
EPOCH:	 6
Loss:			 0.33092
Learning rate:		 0.00279
Accuracy train:		 98.455%
Accuracy validation:	 87.434%
---------------------------

Проверяем точность на тесте

In [20]:
all_accuracies = []
for batch in tqdm(test_iterator):
    all_accuracies.append(accuracy(model(batch.text.permute(1, 0)), batch.label).cpu().numpy())
print(sum(all_accuracies) / len(all_accuracies))

  0%|          | 0/391 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0.8578164961636828


Результат на тестовой части датасета: 

**RNN**: 69%

**GRU**: 85%

Таким образом, мы обучили нейронную сеть распознавать положительные отзывы на фильм и отрицательные с помощью рекурентных нейронных сетей